# lil_cluster
Фундаментальная модель в ансамбле

Изучив специфику авторынка стало понятно, что бренд и модель - названия, не имеюще общих правил. Поэтому модель их может использовать только в качетсве dummy переменных. Но 500 с лишним переменных для меня показались слишком большим количеством для 80 тысяч автомобилей. Поэтому я решил разделить их специфику по моделям. Тут становится интереснее. Оказывается автомобили одной и той же модели, но разных комплектаций могут кратно отличаться в цене. Исходя из этого инсайта я понял, что можно сравнивать автомобили только в рамках одной модели. Этим и будет заниматься lil_cluster

Оказалось, что в датасете множество машин одной и той же модели, более того, в тренировочном датасете оказалось 98% наименований моделей, покрывающие наименования тестового датасета. Исходя из этого, lil_cluster будет иметь главный решающий голос в ансамбле в случаях, когда модель машины ему хорошо известна

In [92]:
%store -r df

In [93]:
df.columns

Index(['bodyType', 'brand', 'color', 'description', 'engineDisplacement',
       'enginePower', 'fuelType', 'mileage', 'model_name', 'numberOfDoors',
       'sell_id', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'price', 'model_product_time', 'car_age'],
      dtype='object')

Для этой модели потребуется сократить разновидности моделей в тренировочной выборки до соответствия их тестовой

In [94]:
import pandas as pd

In [95]:
test = df[df.price==-1]
train = df[df.price!=-1]

In [96]:
print(f'''{test.model_name.nunique()} - количество уникальных моделей 
{len(test.groupby(['brand','model_name']).count())} - количество\
уникальных моделей с привязкой к бренду

названия моделей в тестовом датасете - \
уникальны для всех предоставленных брендов''')

544 - количество уникальных моделей 
544 - количествоуникальных моделей с привязкой к бренду

названия моделей в тестовом датасете - уникальны для всех предоставленных брендов


In [97]:
print(f'''{train.model_name.nunique()} - количество уникальных моделей 
{len(train.groupby(['brand','model_name']).count())} - количество\
уникальных моделей с привязкой к бренду

20 моделей из тренировочного датасета из разных брендов имеют одинаковое название''')

1086 - количество уникальных моделей 
1105 - количествоуникальных моделей с привязкой к бренду

20 моделей из тренировочного датасета из разных брендов имеют одинаковое название


In [98]:
test = df[df.price==-1]
train = df[df.price!=-1]

### Исключает модели, которых нет в  ###
filtered_train = train[train.model_name.isin(test.model_name.unique())
                      ].reset_index(drop=True)



'''Делает неуникальные для разных брендов названия моделей - уникальными'''
###############################################################################
grouped_ft = filtered_train.groupby(['brand','model_name']).count()

valco_non_unique = grouped_ft.reset_index(level=1).model_name.value_counts()[
    grouped_ft.reset_index(level=1).model_name.value_counts()!=1]

def change_name_to_unique(df):
    df_non_unique = df[
        df.model_name.isin(valco_non_unique.index)]

    for i in df_non_unique.index:
        df.at[i,'model_name'] = \
        df.iloc[i].brand + '_' + df.iloc[i].model_name

change_name_to_unique(filtered_train)
change_name_to_unique(test)

### Удаляет модели в трейне, которых нет в тесте после уточнения модели брендом
models_to_drop = (set(filtered_train.model_name.unique()) -set(test.model_name.unique()))
index_to_drop = filtered_train[filtered_train.model_name.isin(models_to_drop)].index

filtered_train.drop(index_to_drop,inplace=True)
###############################################################################


In [99]:
unknown = set(test.model_name.unique())- set(filtered_train.model_name.unique())
display(unknown)
print('18 Моделей из тестового датасета нет в тренировочном')

{'300ZX',
 'ARTEON',
 'COROLLA_II',
 'GOLF_R32',
 'IST',
 'LANCER_RALLIART',
 'LM',
 'OTTI',
 'POLO_GTI',
 'PORTE',
 'PRESEA',
 'RSQ3',
 'SPARKY',
 'TORNEO',
 'WILL_CYPHA',
 'X4_M',
 'Z',
 'ZEST'}

18 Моделей из тестового датасета нет в тренировочном


Модель будет специфичной под тренировочный датасет. Далее я разделяю тестовый датасет, но, важно, чтобы при разделении и соединении датасетов не потерялся порядок, в котором идут авто. 

## Предобработка

In [100]:
for_lil_prediction   = test[~test.model_name.isin(unknown)].reset_index(drop=True)
for_other_prediction = test[test.model_name.isin(unknown)].reset_index(drop=True)

In [104]:
for_model = pd.concat([for_lil_prediction, filtered_train])

In [121]:
preX = pd.concat(
[for_model[['sell_id','price','engineDisplacement', 'enginePower','mileage','car_age', 'model_product_time']] ,
pd.get_dummies(for_model.bodyType,prefix='bodyType'),
pd.get_dummies(for_model.fuelType,prefix='fuelType'),
pd.get_dummies(for_model.vehicleTransmission,prefix='vehicleTransmission'),
pd.get_dummies(for_model.Привод,prefix='Привод'),
pd.get_dummies(for_model.Владельцы,prefix='Владельцы'),
pd.get_dummies(for_model.Владение,prefix='Владение'),
pd.get_dummies(for_model.color,prefix='color'),
pd.get_dummies(for_model.ПТС,prefix='ПТС'),
pd.get_dummies(for_model.Руль,prefix='Руль'),
pd.get_dummies(for_model.numberOfDoors,prefix='numberOfDoors')],axis=1)

preX.sample(5)

test = preX[preX.price==-1]
train = preX[preX.price!=-1]

In [122]:
train

,sell_id,price,engineDisplacement,enginePower,mileage,car_age,model_product_time,bodyType_внедорожник,bodyType_компактвэн,bodyType_купе,...,color_синий,color_чёрный,ПТС_DUPLICATE,ПТС_ORIGINAL,Руль_LEFT,Руль_RIGHT,numberOfDoors_2.0,numberOfDoors_3.0,numberOfDoors_4.0,numberOfDoors_5.0
0,-1,200000.0,2.8,174,350000,29.8,1.0,0,0,0,...,0,1,0,1,1,0,0,0,1,0
1,-1,60000.0,1.8,90,173424,34.8,4.0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
2,-1,99000.0,2.3,136,230000,31.8,1.0,0,0,0,...,1,0,0,1,1,0,0,0,0,1
3,-1,65000.0,1.8,90,240000,31.8,1.0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
4,-1,100000.0,2.0,101,300000,29.8,1.0,0,0,0,...,0,1,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48020,-1,52500000.0,3.0,150,12000,68.8,1.0,0,0,1,...,0,1,0,1,1,0,1,0,0,0
48021,-1,10800000.0,3.0,150,45200,68.8,1.0,0,0,1,...,0,0,0,1,1,0,1,0,0,0
48022,-1,4000000.0,3.0,258,800,2.8,1.0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
48023,-1,3150000.0,2.3,190,15850,2.8,1.0,0,0,0,...,0,0,0,1,1,0,0,0,1,0


In [123]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    train.drop(['price'],axis="columns"), train.price, test_size=0.2, shuffle=True, random_state=42)

def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))*100

In [125]:

from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=5, random_state=0)

regr.fit(X_train, (y_train))

(mape(y_test, (regr.predict(X_test))))*100

3830.196740457426

In [17]:
# Машина
'bodyType', 'engineDisplacement', 'enginePower', 'fuelType', 'vehicleTransmission', 'Привод'

('bodyType', 'engineDisplacement', 'enginePower', 'fuelType')

In [ ]:
# Важное не машина
'car_age', 'mileage'

In [ ]:
# человеческий фактор и маловажное
'color', 'description', 'Владельцы', 'Владение', 'ПТС', 'mileage', 'model_product_time', 'Руль', 'numberOfDoors'

In [18]:
df.columns

Index(['bodyType', 'brand', 'color', 'description', 'engineDisplacement',
       'enginePower', 'fuelType', 'mileage', 'model_name', 'numberOfDoors',
       'sell_id', 'vehicleTransmission', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'price', 'model_product_time', 'car_age'],
      dtype='object')

# Машина

In [49]:
len(filtered_train.price)

47492

In [ ]:
filtered_train[(filtered_train.model_name == '3ER') 

In [85]:
X_test = pd.concat(
[for_lil_prediction[['engineDisplacement', 'enginePower','mileage','car_age', 'model_product_time']] ,
pd.get_dummies(for_lil_prediction.bodyType,prefix='bodyType'),
pd.get_dummies(for_lil_prediction.fuelType,prefix='fuelType'),
pd.get_dummies(for_lil_prediction.vehicleTransmission,prefix='vehicleTransmission'),
pd.get_dummies(for_lil_prediction.Привод,prefix='Привод'),
pd.get_dummies(for_lil_prediction.Владельцы,prefix='Владельцы'),
pd.get_dummies(for_lil_prediction.Владение,prefix='Владение'),
pd.get_dummies(for_lil_prediction.color,prefix='color'),
pd.get_dummies(for_lil_prediction.ПТС,prefix='ПТС'),
pd.get_dummies(for_lil_prediction.Руль,prefix='Руль'),
pd.get_dummies(for_lil_prediction.numberOfDoors,prefix='numberOfDoors'),],axis=1)

X_test['Владельцы_0.0'] = 0

In [78]:
with pd.option_context('display.max_columns', None):
        display(X_test)


,engineDisplacement,enginePower,mileage,car_age,model_product_time,bodyType_внедорожник,bodyType_компактвэн,bodyType_купе,bodyType_лифтбек,bodyType_минивэн,bodyType_пикап,bodyType_седан,bodyType_универсал,bodyType_хэтчбек,bodyType_экстра,fuelType_бензин,fuelType_газ,fuelType_гибрид,fuelType_дизель,fuelType_электро,vehicleTransmission_AUTOMATIC,vehicleTransmission_MECHANICAL,vehicleTransmission_ROBOT,vehicleTransmission_VARIATOR,Привод_задний,Привод_передний,Привод_полный,Владельцы_1.0,Владельцы_2.0,Владельцы_3.0,Владение_0,Владение_1,color_белый,color_зелёный,color_коричневый,color_красный,color_редкий,color_серебристый,color_серый,color_синий,color_чёрный,ПТС_DUPLICATE,ПТС_ORIGINAL,Руль_LEFT,Руль_RIGHT,numberOfDoors_2.0,numberOfDoors_3.0,numberOfDoors_4.0,numberOfDoors_5.0
0,1.2,105,74000,6.8,1.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1
1,1.6,110,60563,3.8,0.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1
2,1.8,152,88000,6.8,1.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1
3,1.6,110,95000,6.8,1.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1
4,1.8,152,58536,8.8,4.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34657,1.6,136,115000,6.8,3.0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0
34658,2.0,190,98000,2.8,2.0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0
34659,2.5,170,360000,23.8,2.0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0
34660,2.0,184,90500,7.8,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1


In [84]:
for_lil_prediction.Владельцы.value_counts()

3.0    15943
1.0     9454
2.0     9265
Name: Владельцы, dtype: int64

In [83]:
df.Владельцы.value_counts()

3.0    51818
2.0    30033
1.0    27690
0.0    11998
Name: Владельцы, dtype: int64

In [80]:
set(X_test.columns) ^  set(X.columns)

{'Владельцы_0.0'}

In [ ]:
c = {"one": 1, "two": 2}
>>> for k,v in c.items():
...    exec(f"tree_{}=%s" % (k,v))


In [86]:
from sklearn import tree                             # Дабы достать дерево

model = tree.DecisionTreeClassifier().fit(X, y)

In [87]:
model.predict(X_test)

C:\Users\zetet\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [59]:
fir name in X.model_name.unique():
    model.fit
    X[X.model_name == name]

array(['100', '200', '80', '90', '920', 'A1', 'A2', 'A3', 'A4',
       'A4_ALLROAD', 'A5', 'A6', 'ALLROAD', 'A7', 'A8', 'AUDI_COUPE',
       'E_TRON', 'Q3', 'Q5', 'Q7', 'Q8', 'R8', 'RS3', 'RS4', 'RS5', 'RS6',
       'RS7', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'SQ5', 'TT', 'TT_RS',
       'TTS', 'AUDI_V8', '02', '1ER', 'M1', '2ER', '2ACTIVETOURER',
       '2GRANDTOURER', '3ER', '321', '326', '340', '4', '5ER', '6ER',
       '7ER', '8ER', 'E3', 'I3', 'I8', 'M2', 'M3', 'M4', 'M5', 'M6', 'X1',
       'X2', 'X3', 'X3_M', 'X4', 'X5', 'X5_M', 'X6', 'X6_M', 'X7', 'Z1',
       'Z3', 'Z3M', 'Z4', 'ATLAS', 'ACCORD', 'ACTY', 'AIRWAVE', 'ASCOT',
       'AVANCIER', 'CITY', 'CIVIC', 'CIVIC_FERIO', 'CIVIC_TYPE_R',
       'CONCERTO', 'CR_V', 'CR_X', 'CR_Z', 'CROSSROAD', 'CROSSTOUR',
       'DOMANI', 'ELEMENT', 'ELYSION', 'FIT', 'FR_V', 'FREED', 'HR_V',
       'INSIGHT', 'INSPIRE', 'INTEGRA', 'JAZZ', 'LEGEND', 'LOGO',
       'MOBILIO', 'MOBILIO_SPIKE', 'N_BOX', 'N_WGN', 'ODYSSEY',
       'ODYSSEY_NA', 'O

In [70]:
y = filtered_train.price


X = pd.concat(
[filtered_train[['engineDisplacement', 'enginePower','mileage','car_age', 'model_product_time']] ,
pd.get_dummies(filtered_train.bodyType,prefix='bodyType'),
pd.get_dummies(filtered_train.fuelType,prefix='fuelType'),
pd.get_dummies(filtered_train.vehicleTransmission,prefix='vehicleTransmission'),
pd.get_dummies(filtered_train.Привод,prefix='Привод'),
pd.get_dummies(filtered_train.Владельцы,prefix='Владельцы'),
pd.get_dummies(filtered_train.Владение,prefix='Владение'),
pd.get_dummies(filtered_train.color,prefix='color'),
pd.get_dummies(filtered_train.ПТС,prefix='ПТС'),
pd.get_dummies(filtered_train.Руль,prefix='Руль'),
pd.get_dummies(filtered_train.numberOfDoors,prefix='numberOfDoors'),],axis=1)

In [39]:
filtered_train[filtered_train.mileage==0].mileage.value_counts()

0    9666
Name: mileage, dtype: int64

In [71]:
X.shape

(47492, 50)

In [42]:
test.mileage.mean()

162009.76788906186

In [41]:
test.mileage.median()

149779.5

In [ ]:
filtered_train.engineDisplacement

In [ ]:
'engineDisplacement' 'enginePower' 

In [ ]:
pd.get_dummies(filtered_train.bodyType)
pd.get_dummies(filtered_train.fuelType)
pd.get_dummies(filtered_train.vehicleTransmission)
pd.get_dummies(filtered_train.Привод)


pd.get_dummies(.Руль)
pd.get_dummies()
pd.get_dummies()

,бензин,газ,гибрид,дизель,электро
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
48020,1,0,0,0,0
48021,1,0,0,0,0
48022,0,0,0,1,0
48023,0,0,0,1,0


In [43]:
pd.concat([filtered_train, pd.get_dummies(filtered_train.bodyType) ],axis=1).corr()

,engineDisplacement,enginePower,mileage,numberOfDoors,sell_id,Владельцы,Владение,price,model_product_time,car_age,внедорожник,компактвэн,купе,лифтбек,минивэн,пикап,седан,универсал,хэтчбек,экстра
engineDisplacement,1.000000,0.807477,0.064723,0.020251,NaN,0.124757,0.048684,0.260147,-0.100185,0.065500,0.274987,-0.086717,0.077090,-0.162766,-0.014414,0.034438,-0.076284,-0.061549,-0.232921,0.030514
enginePower,0.807477,1.000000,-0.192361,-0.003809,NaN,-0.084103,-0.019006,0.520083,-0.167349,-0.208505,0.256620,-0.100668,0.131102,-0.134981,-0.068253,-0.011174,-0.057930,-0.068281,-0.221457,0.009246
mileage,0.064723,-0.192361,1.000000,-0.140845,NaN,0.686333,0.196609,-0.502586,0.027267,0.776211,-0.266019,0.041197,-0.023917,-0.040440,0.078584,0.009006,0.163092,0.156905,0.071124,0.036871
numberOfDoors,0.020251,-0.003809,-0.140845,1.000000,NaN,-0.208310,-0.045481,0.067273,-0.015634,-0.243062,0.610911,0.025025,-0.575404,0.188852,-0.006277,-0.068293,-0.546991,0.134058,0.048608,-0.247755
sell_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Владельцы,0.124757,-0.084103,0.686333,-0.208310,NaN,1.000000,0.248640,-0.521513,-0.017551,0.704696,-0.289953,0.009218,0.049180,-0.030401,0.021626,-0.014056,0.181836,0.099021,0.125980,0.047715
Владение,0.048684,-0.019006,0.196609,-0.045481,NaN,0.248640,1.000000,-0.172090,-0.006619,0.181884,-0.078972,0.008901,0.020975,0.012469,0.034382,0.013024,0.011916,0.037004,0.044222,0.027489
price,0.260147,0.520083,-0.502586,0.067273,NaN,-0.521513,-0.172090,1.000000,-0.061519,-0.476295,0.227985,-0.050649,0.052905,-0.075791,0.019794,-0.001103,-0.113643,-0.083747,-0.129186,0.000206
model_product_time,-0.100185,-0.167349,0.027267,-0.015634,NaN,-0.017551,-0.006619,-0.061519,1.000000,0.069024,-0.027512,0.030271,-0.001936,0.014497,0.082388,0.023124,-0.038176,0.020137,0.009313,0.049941
car_age,0.065500,-0.208505,0.776211,-0.243062,NaN,0.704696,0.181884,-0.476295,0.069024,1.000000,-0.293006,0.041090,0.052320,-0.095559,0.040489,-0.013239,0.177408,0.146834,0.127509,0.082661


,внедорожник,компактвэн,купе,лифтбек,минивэн,пикап,седан,универсал,хэтчбек,экстра
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
48020,0,0,1,0,0,0,0,0,0,0
48021,0,0,1,0,0,0,0,0,0,0
48022,0,0,0,0,0,1,0,0,0,0
48023,0,0,0,0,0,1,0,0,0,0


In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

knn = KNeighborsClassifier(n_neighbors=3)

In [22]:
engineDisplacement

NameError: name 'engineDisplacement' is not defined